# Retrieval Augmented Generation with Dot product distance algorithm using Elastic, Anthropic Claude 3, Amazon Bedrock and Langchain

## Introduction

In this notebook, we will demonstrate how to build a Retrieval Augmented Generation (RAG) solution that uses the dot product retrieval algorithm to enable an AI assistant to provide precise and targeted responses to queries containing specific keywords or phrases related to a company's documentation.


#### Use case

Anycompany is experiencing a high volume of customer queries, many of which are focused on specific topics or contain specific keywords related to the company's products, services, or policies. They have built a conversational AI assistant to answer frequently asked questions, but now they need the AI assistant to be able to provide precise and targeted responses to these queries that contain specific keywords or phrases.

The dot product retrieval algorithm is well-suited for this use case because it can identify sections of the company's documentation that closely match the specific keywords or phrases in the user's query. By leveraging the dot product algorithm, the AI assistant can pinpoint relevant information within the documentation that directly addresses the user's query, enabling it to provide accurate and targeted responses.


#### Persona
You are Bob, an Application Developer at Anycompany. Your task is to enhance the existing conversational AI assistant to handle company-specific queries by incorporating the dot product retrieval algorithm. This will enable the AI assistant to retrieve relevant sections from the company's documentation and provide precise and targeted responses to queries containing specific keywords or phrases from customers.


#### Implementation
In this notebook, we will demonstrate how to create a RAG application using Elasticsearch, Anthropic Claude 3 Sonnet Foundation model, Amazon Bedrock, and Langchain. We will leverage the dot product retrieval algorithm to identify sections from the company's documentation that closely match the specific keywords or phrases in the user's query, enabling our AI assistant to provide precise and targeted responses. 

#### Prerequisites
We're using an Elastic Cloud deployment of Elasticsearch for this notebook. If you don't have an Elastic Cloud deployment, sign up [here](https://cloud.elastic.co/registration) for a free trial.

#### Python 3.10

⚠  For this lab we need to run the notebook based on a Python 3.10 runtime. ⚠


## Installation

To run this notebook you would need to install dependencies - boto3, botocore, elasticsearch and langchain.

In [ ]:
%pip install --upgrade pip
%pip install boto3 --force-reinstall --quiet
%pip install botocore --force-reinstall --quiet
%pip install langchain --force-reinstall --quiet
%pip install langchain-elasticsearch --force-reinstall --quiet
%pip install elasticsearch --force-reinstall --quiet
%pip install unstructured==0.7.12 --force-reinstall --quiet

## Kernel Restart

Restart the kernel with the updated packages that are installed through the dependencies above

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

## Setup 

Import the necessary libraries

In [ ]:
import json
import os
import sys
import boto3
import botocore
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models.bedrock import BedrockChat
from langchain.embeddings import BedrockEmbeddings
from botocore.client import Config
from langchain_community.retrievers import AmazonKnowledgeBasesRetriever
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_elasticsearch import ElasticsearchStore
from elasticsearch import Elasticsearch
from langchain.schema.runnable import RunnablePassthrough
from langchain.chains import RetrievalQA
from getpass import getpass
from langchain.prompts import PromptTemplate
from langchain.document_loaders import DirectoryLoader
from pathlib import Path

## Initialization

Initiate Bedrock Runtime and BedrockChat

In [ ]:
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime')

modelId = 'anthropic.claude-3-sonnet-20240229-v1:0' # change this to use a different version from the model provider
embeddingmodelId = 'amazon.titan-embed-text-v1' # change this to use a different embedding model

llm = BedrockChat(model_id=modelId, client=bedrock_client)
embeddings = BedrockEmbeddings(model_id=embeddingmodelId,client=bedrock_client, normalize=True) #for using dot product we need to normalize embeddings to unit vectors

## Read files from directory

Load all PDF files which are present in the directory

In [ ]:
TMP_DIR = os.path.join(os.path.dirname(os.path.realpath('__file__')), 'media')
loader = DirectoryLoader(TMP_DIR, glob='**/*.pdf')
documents = loader.load()

## Split Documents

Chunk documents into passages in order to improve the retrieval specificity and to ensure that we can provide multiple passages within the context window of the final question answering prompt.

Here we are chunking documents into 1000 token passages with an overlap of 0 tokens.

Here we are using Recursive Character Text splitter but Langchain offers more advanced splitters to reduce the chance of context being lost.

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
        separators=['\n\n', '\n', '.', ','],
        chunk_size=1000,
        chunk_overlap=0
        )
texts = text_splitter.split_documents(documents)

## Connect to Elasticsearch

We'll use the Cloud ID to identify our deployment, because we are using Elastic Cloud deployment. To find the Cloud ID for your deployment, go to [Cloud ID](https://cloud.elastic.co/deployments) and select your deployment.

We will use ElasticsearchStore to connect to our elastic cloud deployment. This would help create and index data easily. 

In [ ]:
cloud_id = getpass("Elastic deployment Cloud ID: ")
cloud_api_key = getpass("Elastic deployment API Key: ")
index_name= "new-index-1"

vector_store = ElasticsearchStore(
        es_cloud_id=cloud_id,  
        index_name= index_name, 
        embedding=embeddings,
        es_api_key=cloud_api_key)

## Index data into Elasticsearch and initialize retriever

Next, we will index data to elasticsearch using ElasticsearchStore.from_documents. We will use Cloud ID, Password and Index name values set in the Create cloud deployment step. We will set embedding to BedrockEmbeddings to embed the texts.

In a RAG application, the goal is to retrieve relevant information from a knowledge base or corpus to assist in the generation task. The relevance of a piece of information is often determined by its similarity to the input query or context. 

Elasticsearch supports the following vector distance similarity algorithms:
* cosine
* euclidean
* dot_product

The cosine similarity algorithm is the default.

You can also use Dot product as a similarity measure.

While both the dot product and cosine similarity measure the similarity between vectors, the dot product takes into account the magnitudes and directions of the vectors, while cosine similarity focuses solely on the angle between the vectors, making it more suitable for applications involving high-dimensional sparse vectors, such as text analysis and information retrieval.

For example, imagine you're building a music recommendation system where users explicitly rate songs on a scale of 1 to 5. In this scenario, using the dot product to calculate similarity between user vectors (representing their ratings for different songs) and item vectors (representing features of each song) can effectively capture the strength of user-item interactions.By computing the dot product, you're essentially measuring how much two vectors align in the same direction, indicating stronger preferences or interactions between users and items. This approach works well when the ratings are explicit and the magnitude of the ratings matters in determining similarity.However, if the ratings are sparse or noisy, and the focus is more on the direction of preferences rather than the magnitude, cosine similarity might be more suitable. Cosine similarity normalizes vectors and focuses solely on the direction of the vectors, which can be beneficial in scenarios where the magnitude of ratings varies significantly among users or items.

In [ ]:
vectordb = vector_store.from_documents(
        texts, 
        embeddings,
        index_name=index_name,
        es_cloud_id=cloud_id,
        es_api_key=cloud_api_key,
        distance_strategy="DOT_PRODUCT"
        )

retriever = vectordb.as_retriever()

## Model Invocation and Response Generation using RetrievalQA chain

Now that we have the passages stored in Elasticsearch and LLM is initialized, we can now ask a question to get the relevant passages.

In [ ]:
query = "What is an important first step for machine learning metamodeling?"


machinelearning_advisor_template = """
    Human: You will be acting as a Machine Learning advisor on complex Mechanical systems named Poly created by the company Polymath. 
    Your goal is to give advice related to Application of Machine Learning on Mechanical Systems to users. You will be replying to users 
    who are asking questions on Application of Machine Learning in Mechanical Systems 
    site and who will be confused if you don't respond in the character of Poly.
    
    You should maintain a friendly customer service tone.

    Here is the document you should reference when answering the user: <context>{context}</context>

    Here are some important rules for the interaction:
    - Always stay in character, as Poly, a Machine Learning advisor on complex Mechanical systems
    - If you are unsure how to respond, say “Sorry, I didn’t understand that. Could you repeat the question?”
    - If someone asks something irrelevant, say, “Sorry, I am Poly and I give career advice. Do you have a question related to Application of Machine Learning in Mechanical Systems today I can help you with?”

    Here is an example of how to respond in a standard interaction:

    <example>
    User: Hi, how were you created and what do you do?
    Poly: Hello! My name is Poly, and I was created by Polymath  to give advice on Application of Machine Learning in Mechanical Systems. 
        What can I help you with today?
    User: Hi, how can I use Decision Trees?
    Poly: The method for using decision trees described in the given context is to first rank all
        all the rules within the decision tree to find the most meaningful and reliable design subspaces.
        Then, determine how many significant rules exist within the dataset to present the engineer
        with the most reliable and important knowledge. Finally, use the resulting rules from the decision tree
        to make predictions or decisions based on the given target variable.
        User: Hi, What is Machine Learning?
    Poly: Machine learning is a tool that allows predictions about future behavior to be drawn from existing
        data sets. It is used in everything from spam filters to self-driving cars. While the field has been
        around for decades, it has flourished over the last several years as computing power has increased
        and user-friendly toolkits have been developed in a variety of programming languages. We do not
        go into detail on machine learning theory and practice in this paper, but we do introduce the
        concepts necessary for understanding our implementation of it.
    </example>

    Here is the user’s question: <question> {question} </question>

    How do you respond to the user’s question?
    Think about your answer first before you respond. Put your response in <response></response> tags.
    Assistant: <response>"""

prompt = PromptTemplate(template=machinelearning_advisor_template, input_variables=["context","question"])
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff",retriever=retriever, return_source_documents=True, chain_type_kwargs={"prompt": prompt})
response = qa_chain.invoke(query)
print(response["result"])

## Delete Elasticsearch Index

Delete the Elasticsearch index

In [ ]:
es = Elasticsearch(cloud_id=cloud_id, api_key=cloud_api_key)
es.options(ignore_status=[400,404]).indices.delete(index=index_name)

## Conclusion
You have now experimented with `langchain` SDK to get an exposure to RAG Application building using Elastic, Anthropic Claude 3 and Amazon Bedrock API. Using langchain you have asked questions on your domain specicific documents.

### Take aways
- Adapt this notebook to experiment with different Claude 3 models available through Amazon Bedrock. 
- Change the prompts to your specific usecase and evaluate the output of different models.
- Play with the token length to understand the latency and responsiveness of the service.
- Apply different prompt engineering principles to get better outputs.

## Thank You